In [1]:
import tensorflow as tf 
print('tensorflow version', tf.__version__)


2023-05-04 12:06:38.134794: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensorflow version 2.12.0


In [2]:
x = [[3.]]
y = [[4.]]
print('Result: {}'.format(tf.matmul(x, y)))

Result: [[12.]]
